In [1]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.
country = "ML"
contigs = ["3L"]
analysis_version = "2025.02.13"
dask_scheduler = "single-threaded"
cohorts_analysis = "20240924"

In [2]:
# Parameters
analysis_version = "2025.02.13"
min_cohort_size = 15
max_cohort_size = 70
sample_sets = ["3.0"]
sample_query = (
    "unrestricted_use and taxon in ['gambiae', 'coluzzii', 'arabiensis', 'bissau']"
)
contigs = ["2RL", "3RL", "X"]
cohorts_analysis = "20240924"
h12_calibration_contig = "3L"
h12_signal_detection_min_delta_aic = 1000
h12_signal_detection_min_stat_max = 0.1
h12_signal_detection_gflanks = [6]
dask_scheduler = "single-threaded"
alerts = ["01", "02", "03", "04", "05", "06", "07", "08", "09"]
country = "CM"


In [3]:
from pyprojroot import here

root = here()
%run -i {root}/workflow/common/scripts/setup.py
%run -i {root}/workflow/site/scripts/page-setup.py

# N.B., do not add the "remove-output" tag to this cell!!! If you do,
# the bokeh javascript libraries will not get loaded in the generated
# HTML page. The call to output_notebook() injects javascript in the
# cell output which triggers the bokeh javascript libraries to be loaded
# in the page.
output_notebook(hide_banner=True)

# Cameroon

The map below shows regions where mosquito genomic data are available and genome-wide 
selection scans have been performed.

Click on a region to see more information about mosquito cohorts for which data are 
available.

In [4]:
gdf_cohorts_country = gdf_cohorts.query(f"country_alpha2 == '{country}'")

In [5]:
from ipyleaflet import Map, Marker, basemaps, AwesomeIcon
from ipywidgets import HTML

center = gdf_cohorts[["latitude", "longitude"]].mean().to_list()
m = Map(center=center, zoom=7, basemap=basemaps.OpenTopoMap)

for shapeID in gdf_cohorts_country.shapeID.unique():
    if shapeID is None:
        continue
    df = gdf_cohorts.query("shapeID == @shapeID")
    html_text = "<br>".join(
        [
            f'<a href="../cohort/{coh_id}.html">{row.cohort_label} (n={row.cohort_size})</a>'
            for coh_id, row in df.iterrows()
        ]
    )

    lat, long = (
        df[["representative_lat", "representative_lon"]].drop_duplicates().values[0]
    )

    icon1 = AwesomeIcon(name="bug", marker_color="gray", icon_color="white", spin=False)

    marker = Marker(
        location=(lat, long), icon=icon1, draggable=False, opacity=0.7, color="gray"
    )
    m.add_layer(marker)
    message2 = HTML()
    message2.value = html_text
    marker.popup = message2

m

Map(center=[4.152162368757072, 8.621838379566977], controls=(ZoomControl(options=['position', 'zoom_in_text', …

<br></br>

In [6]:
def make_clickable(url, name):
    return '<a href="{}" rel="noopener noreferrer" target="_blank">{}</a>'.format(
        url, name
    )

In [7]:
gdf_cohorts = gdf_cohorts_country.reset_index()
gdf_cohorts["url"] = "../cohort/" + gdf_cohorts["cohort_id"] + ".html"
gdf_cohorts["cohort_label"] = gdf_cohorts.apply(
    lambda x: make_clickable(x["url"], x["cohort_label"]), axis=1
)

gdf_cohorts = gdf_cohorts[
    [
        "cohort_label",
        "cohort_size",
        "admin1_name",
        "admin2_name",
        "taxon",
        "year",
        "quarter",
    ]
]
gdf_cohorts = gdf_cohorts.rename(columns={"cohort_label": "Cohort label"}).set_index(
    "Cohort label"
)
gdf_cohorts.columns = [
    "Cohort size",
    "Admin1 area",
    "Admin2 area",
    "Taxon",
    "Year",
    "Quarter",
]
gdf_cohorts.style

,Cohort size,Admin1 area,Admin2 area,Taxon,Year,Quarter
Cohort label,,,,,,
Cameroon / Mayo-Kani / gambiae / 2005,18,Far North,Mayo-Kani,gambiae,2005,-1
Cameroon / Haut-Nyong / gambiae / 2009 / Q3,95,East,Haut-Nyong,gambiae,2009,3
Cameroon / Lom-Et-Djerem / gambiae / 2009 / Q3,163,East,Lom-Et-Djerem,gambiae,2009,3
Cameroon / Noun / gambiae / 2013 / Q4,15,West,Noun,gambiae,2013,4
